<a href="https://colab.research.google.com/github/Risskr/Stock-App/blob/Working/StocksApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Section 1: Set Up**


In [1]:
# #-----refesh python script-------#
# get_ipython().kernel.do_shutdown(restart=True)

# #----GPU Setup------#
# #Check GPUs
# !nvidia-smi

# #load cudf to use GPUs for analysis
# %load_ext cudf.pandas


#------Imports--------#
import pandas as pd
import datetime
import pickle
import numpy as np
import requests
import time
import os


#-------Froms-------#
from tqdm.notebook import tqdm
from datetime import datetime, timedelta


# #------Force Connect MyDrive-------#
# from google.colab import drive
# drive.mount('/content/drive')


# **Section 2: Stock Data**

## Ensure the last 6 months of EODHD Data is available

In [2]:
# Ensure the last 6 months of EODHD Data is available
"""
Returns:
nasdaq_df:
  <class 'pandas.core.frame.DataFrame'>
  RangeIndex: 627225 entries, 0 to 627224
  Data columns (total 8 columns):
  #   Column          Non-Null Count   Dtype
  ---  ------          --------------   -----
  0   date            627225 non-null  datetime64[ns]
  1   ticker          627098 non-null  object
  2   open            627225 non-null  float64
  3   high            627225 non-null  float64
  4   low             627225 non-null  float64
  5   close           627225 non-null  float64
  6   adjusted_close  627225 non-null  float64
  7   volume          627225 non-null  float64
"""
#API_KEY = 'demo'  # Replace with your API key if not using demo
API_KEY = '68433aff09ea73.10710364'
EXCHANGE = 'NASDAQ'
DAYS_BACK = 180  # Approx. 6 months
SAVE_PATH = '/content/drive/MyDrive/Colab Notebooks/nasdaq_bulk_eod.csv'
MAX_CALLS_PER_RUN = 200  # Use 1–5 for the free tier
SECONDS_BETWEEN_CALLS = 0  # Add delay to be respectful

# Generate past 6 months of weekdays
today = datetime.utcnow().date()
dates = [today - timedelta(days=i) for i in range(DAYS_BACK)]
dates = sorted([d for d in dates if d.weekday() < 5])  # Keep only weekdays

# Load already-downloaded dates if file exists
downloaded_dates = set()
if os.path.exists(SAVE_PATH):
    df_existing = pd.read_csv(SAVE_PATH)
    downloaded_dates = set(pd.to_datetime(df_existing['date']).dt.date)
# Filter to only dates we haven’t downloaded
pending_dates = [d for d in dates if d not in downloaded_dates]

# Prepare data holder
all_data = []

for i, date in enumerate(pending_dates[:MAX_CALLS_PER_RUN]):
    date_str = date.strftime('%Y-%m-%d')
    url = f'https://eodhd.com/api/eod-bulk-last-day/{EXCHANGE}?api_token={API_KEY}&fmt=json&date={date_str}'
    print(f"[{i+1}] Fetching {date_str}...")

    try:
        response = requests.get(url)
        response.raise_for_status()
        day_data = response.json()

        for entry in day_data:
            all_data.append({
                'date': entry.get('date'),
                'ticker': entry.get('code'),
                'open': entry.get('open'),
                'high': entry.get('high'),
                'low': entry.get('low'),
                'close': entry.get('close'),
                'adjusted_close': entry.get('adjusted_close'),
                'volume': entry.get('volume'),
            })

        # Respect the delay
        if i < MAX_CALLS_PER_RUN - 1:
            time.sleep(SECONDS_BETWEEN_CALLS)

    except Exception as e:
        print(f"⚠️ Error on {date_str}: {e}")

# Append or save the new data
if all_data:
    df_new = pd.DataFrame(all_data)
    if os.path.exists(SAVE_PATH):
        df_existing = pd.read_csv(SAVE_PATH)
        df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    else:
        df_combined = df_new

    df_combined.to_csv(SAVE_PATH, index=False)
    print(f"✅ Data for {len(all_data)} entries added to {SAVE_PATH}")
else:
    print("⚠️ No new data fetched.")

# Import CSV into code
nasdaq_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/nasdaq_bulk_eod.csv")

⚠️ No new data fetched.


## Get list of Tickers with Min Market Cap and populate screener_data_df

In [3]:
# Get list of Tickers with a min market cap and a list of common stocks
"""
Returns:
screener_data_df:
  <class 'pandas.core.frame.DataFrame'>
  RangeIndex: 262 entries, 0 to 261
  Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype
  ---  ------                 --------------  -----
 0   code                   262 non-null    object
 1   name                   262 non-null    object
 2   last_day_data_date     262 non-null    object
 3   adjusted_close         262 non-null    float64
 4   refund_1d              262 non-null    float64
 5   refund_1d_p            262 non-null    float64
 6   refund_5d              262 non-null    float64
 7   refund_5d_p            262 non-null    float64
 8   exchange               262 non-null    object
 9   currency_symbol        262 non-null    object
 10  market_capitalization  262 non-null    int64
 11  earnings_share         262 non-null    float64
 12  dividend_yield         145 non-null    float64
 13  sector                 262 non-null    object
 14  industry               262 non-null    object
 15  avgvol_1d              262 non-null    int64
 16  avgvol_200d            262 non-null    float64
 17  Country                262 non-null    object
 18  Exchange               262 non-null    object
 19  Currency               262 non-null    object
 20  Type                   262 non-null    object
 21  last_day_change        262 non-null    float64

"""

# -------------------- CONFIG --------------------
API_KEY = '68433aff09ea73.10710364'  # Replace with your EODHD key
MIN_MARKET_CAP = 10_000_000_000  # Changeable: $1B, $10B, etc.
EXCHANGE = 'NASDAQ'
RESULTS_PER_PAGE = 500  # Max per EODHD API
# ------------------------------------------------

def get_filtered_nasdaq_stocks(api_key, min_cap, exchange="NASDAQ"):
    all_data = []
    offset = 0

    while True:
        url = (
            "https://eodhd.com/api/screener"
            f"?api_token={api_key}"
            f"&filters=["
            f'["exchange","=","{exchange}"],'
            #f'["type","=","Common Stock"],'
            f'["market_capitalization",">=",{min_cap}]'
            f"]"
            f"&sort=market_capitalization.desc"
            f"&limit={RESULTS_PER_PAGE}&offset={offset}&fmt=json"
        )

        response = requests.get(url)
        result = response.json()
        batch = result.get("data", [])

        if not batch:
            break

        all_data.extend(batch)
        offset += RESULTS_PER_PAGE

    return pd.DataFrame(all_data)

# Run filter to get screener_data_df
screener_data_df = get_filtered_nasdaq_stocks(API_KEY, MIN_MARKET_CAP)

# Get exchange symbol list with type
meta_url = f'https://eodhd.com/api/exchange-symbol-list/NASDAQ?api_token={API_KEY}&fmt=json'
meta_df = pd.DataFrame(requests.get(meta_url).json())
common_df = meta_df[meta_df['Type'] == 'Common Stock'].copy() # Add .copy() to avoid SettingWithCopyWarning

# Combine screener_data_df with relevant columns from common_df
screener_data_df = pd.merge(
    screener_data_df,
    common_df[['Code', 'Country', 'Exchange', 'Currency', 'Type']],
    left_on='code',
    right_on='Code',
    how='inner'
).drop('Code', axis=1)

# --- Calculate Daily Change ---

if 'nasdaq_df' in locals() and not nasdaq_df.empty:
    # Ensure date is datetime
    nasdaq_df['date'] = pd.to_datetime(nasdaq_df['date'])

    # Sort and calculate daily change on the original nasdaq_df
    temp_nasdaq_sorted = nasdaq_df.sort_values(by=['ticker', 'date']).copy()
    temp_nasdaq_sorted['daily_change'] = temp_nasdaq_sorted.groupby('ticker')['adjusted_close'].pct_change()

    # Get the last day's data from the temp sorted dataframe
    last_date = temp_nasdaq_sorted['date'].max()
    last_day_data = temp_nasdaq_sorted[temp_nasdaq_sorted['date'] == last_date]
    last_day_changes = last_day_data.set_index('ticker')['daily_change'].to_dict()

    # Append last_day_changes to screener_data_df
    screener_data_df['last_day_change'] = screener_data_df['code'].map(last_day_changes)
    # Fill potential NaN values if some tickers in screener_data_df didn't have data on the last day
    screener_data_df['last_day_change'] = screener_data_df['last_day_change'].fillna(0)
else:
    print("Warning: nasdaq_df not found or empty. Cannot calculate and append last_day_change.")

## Filter nasdaq data for screener_data_df and type: common

In [4]:
# prompt: I want to filter the Nasdaq stocks to only include tickers that are part of the common_df and the min_market_cap_df. Same this df as a new variable
# Load your EOD data
from datetime import datetime, timedelta
# nasdaq_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/nasdaq_bulk_eod.csv")
nasdaq_df['date'] = pd.to_datetime(nasdaq_df['date'])

# Now filter nasdaq_df using the combined dataframe and type 'Common Stock'
filtered_nasdaq_df = nasdaq_df[
    nasdaq_df['ticker'].isin(screener_data_df[screener_data_df['Type'] == 'Common Stock']['code'])
]

## Function: Correlation Coeficient for entire stock data set

In [5]:
#return six month and three month spearman correlations for all unique pairs of stocks

import pandas as pd
from datetime import datetime # Import datetime
from tqdm.notebook import tqdm # Import tqdm

# ## Function: Correlation Coeficient for entire stock data set
def calculate_lagged_correlation(df, lag_days, range_months):
  """
  Calculates the pairwise spearman correlation coefficient between all stocks
  in a DataFrame for a specified period with a given lag.
  The start date is calculated by subtracting range_months from today's date.

  Args:
    df: DataFrame with 'date', 'ticker', and 'adjusted_close' columns.
    lag_days: The number of days to lag the second stock's data.
    range_months: The number of months to include in the analysis period.

  Returns:
    correlation_matrix: A pandas DataFrame with the following
      Index: Tickers
      Columns: Tickers
      Values: Spearman correlation coefficients between stocks
  """
  # Calculate the end date (today's date)
  end_datetime = datetime.now()

  # Calculate the start date for the specified period by subtracting range_months
  start_datetime = end_datetime - pd.DateOffset(months = range_months)


  # Ensure the 'date' column is in datetime format
  df['date'] = pd.to_datetime(df['date'])

  # Filter the DataFrame for the specified date range
  filtered_df = df[(df['date'] >= start_datetime) & (df['date'] <= end_datetime)].copy()

  # Filter out rows where volume is 0
  filtered_df = filtered_df[filtered_df['volume'] > 0].copy()

  # Get unique tickers in the filtered data
  tickers = filtered_df['ticker'].unique()

  #breakpoint()

  # Create an empty DataFrame to store correlation results
  correlation_matrix = pd.DataFrame(index=tickers, columns=tickers, dtype=float)

  # Iterate through all pairs of tickers with a progress bar
  for ticker_a in tqdm(tickers, desc="Calculating correlations"): # Add tqdm here

    # Extract data for each ticker and name the series for clarity
    stock_a_data = filtered_df[filtered_df['ticker'] == ticker_a].set_index('date')['adjusted_close']
    #breakpoint()

    for ticker_b in tickers:
      if ticker_a != ticker_b:
        # Extract data for each ticker and name the series for clarity
        stock_b_data = filtered_df[filtered_df['ticker'] == ticker_b].set_index('date')['adjusted_close']
        #breakpoint()

        # Align the dataframes based on the date index
        # Suffixes will be applied to the 'adjusted_close' column name
        aligned_data = pd.merge(stock_a_data, stock_b_data,
                                left_index=True, right_index=True,
                                how='inner', suffixes=('_A', '_B'))

        # Apply the lag to stock_b_data, referencing the suffixed column name
        lagged_stock_b_data = aligned_data['adjusted_close_B'].shift(lag_days)

        #breakpoint()

        # Calculate correlation, dropping NaN values
        # Using Spearman method and minimum periods
        #correlation = aligned_data['adjusted_close_A'].corr(lagged_stock_b_data, method='spearman', min_periods=100)

        correlation = aligned_data['adjusted_close_A'].corr(lagged_stock_b_data, method='spearman')


        # Store the correlation in the matrix
        correlation_matrix.loc[ticker_a, ticker_b] = correlation

  return correlation_matrix


In [6]:
# Check GoogleDrive for today's files, in not run the correlation function
"""
Returns:
  three_month_spearman_lagged_correlations and six_month_spearman_lagged_correlations: A pandas DataFrame with the following
    Index: Tickers
    Columns: Tickers
    Values: Spearman correlation coefficients between stocks
"""
import pandas as pd
import os
from datetime import datetime, timedelta

# Determine the date to use for file checking (today or the previous Friday)
today = datetime.now().date()
if today.weekday() >= 5:  # 5 is Saturday, 6 is Sunday
    # It's a weekend, find the most recent Friday
    days_since_friday = today.weekday() - 4  # 5 - 4 = 1 for Sat, 6 - 4 = 2 for Sun
    file_date = today - timedelta(days=days_since_friday)
else:
    # It's a weekday, use today's date
    file_date = today

file_date_str = file_date.strftime('%Y%m%d')

# Define the file paths in Google Drive using the determined date
three_month_file = f'/content/drive/MyDrive/Colab Notebooks/Correlation_Data/{file_date_str}_three_month_spearman_lagged_correlation.csv'
six_month_file = f'/content/drive/MyDrive/Colab Notebooks/Correlation_Data/{file_date_str}_six_month_spearman_lagged_correlation.csv'

# Check if files exist and load them
try:
    three_month_spearman_lagged_correlations = pd.read_csv(three_month_file, index_col=0)
    print(f"Loaded existing file: {three_month_file}")
    six_month_spearman_lagged_correlations = pd.read_csv(six_month_file, index_col=0)
    print(f"Loaded existing file: {six_month_file}")

except FileNotFoundError:
    print("One or both correlation files not found. Calculating correlations...")
    # Calculate correlations if files don't exist
    # Ensure 'filtered_nasdaq_df' is defined from the preceding code
    if 'filtered_nasdaq_df' in locals():
        three_month_spearman_lagged_correlations = calculate_lagged_correlation(filtered_nasdaq_df, lag_days=1, range_months=3)
        six_month_spearman_lagged_correlations = calculate_lagged_correlation(filtered_nasdaq_df, lag_days=1, range_months=6)

        # Save the newly calculated correlations to Google Drive
        # Use today's date for saving new files
        today_str = datetime.now().strftime('%Y%m%d')
        new_three_month_file = f'/content/drive/MyDrive/Colab Notebooks/Correlation_Data/{today_str}_three_month_spearman_lagged_correlation.csv'
        new_six_month_file = f'/content/drive/MyDrive/Colab Notebooks/Correlation_Data/{today_str}_six_month_spearman_lagged_correlation.csv'

        three_month_spearman_lagged_correlations.to_csv(new_three_month_file)
        print(f"Saved calculated file: {new_three_month_file}")
        six_month_spearman_lagged_correlations.to_csv(new_six_month_file)
        print(f"Saved calculated file: {new_six_month_file}")
    else:
        print("Error: 'filtered_nasdaq_df' is not defined. Please ensure the preceding code ran correctly.")

Loaded existing file: /content/drive/MyDrive/Colab Notebooks/Correlation_Data/20250613_three_month_spearman_lagged_correlation.csv
Loaded existing file: /content/drive/MyDrive/Colab Notebooks/Correlation_Data/20250613_six_month_spearman_lagged_correlation.csv


## Process correlated Data and get gravitational scores

In [43]:
import pandas as pd
import numpy as np

# Function to process stock correlation data, calculate gravitational forces,
# and filter connections for visualization based on the force.
def process_and_score_stocks(
    six_month_correlations,
    three_month_correlations,
    screener_data_df,
    source_ticker,
    min_nodes,
    max_nodes,
    threshold_percent
):
    """
    Processes stock correlation data for a specific source ticker.
    It filters for positive correlations, computes a dynamic impact score (gravitational_force),
    filters connections, and then calculates a final net gravitational force and the
    maximum potential force under ideal conditions.

    Args:
      six_month_correlations: The six-month spearman lagged correlation matrix.
      three_month_correlations: The three-month spearman lagged correlation matrix.
      screener_data_df: DataFrame with additional stock information.
      source_ticker: The ticker symbol for which to process data.
      min_nodes: Minimum number of correlated stocks to return.
      max_nodes: Maximum number of correlated stocks to return.
      threshold_percent: A percentage (0.0 to 1.0) of the maximum force to use as a filter.

    Returns:
      processed_data_df: A pandas DataFrame with processed data for visualization.
      net_gravitational_force: The actual calculated force for the source ticker,
                               representing the sum of the raw, signed gravitational forces.
      max_potential_force: The theoretical maximum force if all filtered stocks had a
                           perfect 1.0 correlation and a positive daily change. This
                           provides a baseline for what is theoretically possible.
                           Comparing net_gravitational_force to this value gives context
                           to the strength and alignment of the force.
    """
    # --- Data Unpivoting and Initial Setup ---
    # Start with the 6-month correlation data as the base
    correlation_df = six_month_correlations.rename_axis('source', axis=0)
    grouped_correlation_data = correlation_df.stack().reset_index()
    grouped_correlation_data.columns = ['source', 'target', 'six_month_spearman_correlation']

    grouped_correlation_data = grouped_correlation_data[
        (grouped_correlation_data['source'] != grouped_correlation_data['target']) &
        (grouped_correlation_data['target'] != source_ticker)
    ].copy()

    # --- Filter for the specific source ticker ---
    source_connections = grouped_correlation_data[grouped_correlation_data['source'] == source_ticker].copy()
    if source_connections.empty:
        print(f"No correlation data found for source ticker {source_ticker}.")
        return pd.DataFrame(), 0.0, 0.0

    # We only care about positively correlated stocks for this model
    positive_corr_group = source_connections[source_connections['six_month_spearman_correlation'] > 0].copy()
    if positive_corr_group.empty:
        print(f"No positive correlations found for source ticker {source_ticker}.")
        return pd.DataFrame(), 0.0, 0.0

    # --- Enrich Data (before filtering) ---
    # Add 3-month correlation data
    positive_corr_group['three_month_spearman_correlation'] = positive_corr_group.apply(
        lambda row: three_month_correlations.loc[row['source'], row['target']] if row['source'] in three_month_correlations.index and row['target'] in three_month_correlations.columns else 0, axis=1
    )

    # Add market data
    screener_cols_to_add = ['code', 'market_capitalization', 'last_day_change']
    required_screener_cols = ['code', 'market_capitalization', 'last_day_change']
    if not all(col in screener_data_df.columns for col in required_screener_cols):
        missing = [col for col in required_screener_cols if col not in screener_data_df.columns]
        raise ValueError(f"screener_data_df is missing required columns: {missing}")

    screener_info = screener_data_df[screener_cols_to_add].rename(columns={'code': 'target'})
    positive_corr_group = pd.merge(positive_corr_group, screener_info, on='target', how='left')
    positive_corr_group.dropna(subset=['market_capitalization', 'last_day_change'], inplace=True)
    if positive_corr_group.empty:
        print(f"No valid connections after merging screener data for {source_ticker}.")
        return pd.DataFrame(), 0.0, 0.0

    # --- Calculate Dynamic Impact Score (Gravitational Force) ---
    epsilon = 1e-9 # Small value to avoid log(0) issues.
    # Weights for recency bias
    w_3m = 0.6
    w_6m = 0.4
    # **NEW NAME** "unified_correlation" is a weighted average of recent correlations.
    positive_corr_group['unified_correlation'] = (
        w_3m * positive_corr_group['three_month_spearman_correlation'] +
        w_6m * positive_corr_group['six_month_spearman_correlation']
    )

    # **NEW SCALE** Calculate a market cap influence score scaled between 0 and 1.
    positive_corr_group['Market Cap'] = positive_corr_group['market_capitalization']
    log_caps = np.log(positive_corr_group['Market Cap'].clip(lower=epsilon)) # **REMOVED** market_cap_offset

    min_log_cap = log_caps.min()
    max_log_cap = log_caps.max()
    log_cap_range = max_log_cap - min_log_cap

    # Scale influence from 0 (smallest cap) to 1 (largest cap).
    if log_cap_range > 0:
        positive_corr_group['market_cap_influence'] = (log_caps - min_log_cap) / log_cap_range
    else:
        positive_corr_group['market_cap_influence'] = 0.5 # Neutral value if all caps are the same

    # The `gravitational_force` is a product of recent correlation strength and market influence.
    positive_corr_group['gravitational_force'] = (
        positive_corr_group['unified_correlation'] *
        positive_corr_group['market_cap_influence'] *
        100
    )

    # --- Apply Filtering ---
    max_abs_force = positive_corr_group['gravitational_force'].abs().max()
    if pd.isna(max_abs_force) or max_abs_force == 0:
        return pd.DataFrame(), 0.0, 0.0

    force_threshold = max_abs_force * threshold_percent
    filtered_by_force_threshold = positive_corr_group[positive_corr_group['gravitational_force'].abs() >= force_threshold]

    # Enforce min/max node constraints
    if len(filtered_by_force_threshold) < min_nodes:
        final_filtered_df = positive_corr_group.sort_values(by='gravitational_force', key=abs, ascending=False).head(min_nodes).copy()
    elif len(filtered_by_force_threshold) > max_nodes:
        final_filtered_df = filtered_by_force_threshold.sort_values(by='gravitational_force', key=abs, ascending=False).head(max_nodes).copy()
    else:
        final_filtered_df = filtered_by_force_threshold.copy()

    if final_filtered_df.empty:
        print(f"No connections remained for {source_ticker} after filtering.")
        return pd.DataFrame(), 0.0, 0.0

    # --- Calculate Final Net Force and Visualization Parameters ---
    final_filtered_df['Daily Change'] = final_filtered_df['last_day_change'] * 100

    final_filtered_df['signed_gravitational_force'] = final_filtered_df.apply(
        lambda row: row['gravitational_force'] if row['Daily Change'] >= 0 else -row['gravitational_force'],
        axis=1
    )

    net_gravitational_force = final_filtered_df['signed_gravitational_force'].sum()
    max_potential_force = final_filtered_df['market_cap_influence'].sum() * 100

    # --- Calculate Visualization Parameters ---
    min_corr, max_corr = final_filtered_df['unified_correlation'].min(), final_filtered_df['unified_correlation'].max()
    corr_range = max_corr - min_corr if max_corr > min_corr else 1.0
    final_filtered_df['Orbital Radius'] = (final_filtered_df['unified_correlation'] - min_corr) / corr_range

    log_caps_filtered = np.log(final_filtered_df['Market Cap'].clip(lower=epsilon))
    min_log_cap_filtered, max_log_cap_filtered = log_caps_filtered.min(), log_caps_filtered.max()
    log_cap_range_filtered = max_log_cap_filtered - min_log_cap_filtered

    if log_cap_range_filtered > 0:
        final_filtered_df['Planet Radius'] = (log_caps_filtered - min_log_cap_filtered) / log_cap_range_filtered
    else:
        final_filtered_df['Planet Radius'] = 0.5

    # --- Final Cleanup and Column Selection ---
    # **NEW NAME** "gravitational_impact" shows the relative % contribution of each stock.
    final_filtered_df['gravitational_percent'] = (final_filtered_df['signed_gravitational_force'] / final_filtered_df['gravitational_force'].sum()) * 100

    final_columns = [
        'source', 'target', 'Daily Change', 'six_month_spearman_correlation',
        'three_month_spearman_correlation', 'unified_correlation',
        'Orbital Radius', 'Market Cap', 'Planet Radius', 'market_cap_influence',
        'gravitational_force', 'signed_gravitational_force', 'gravitational_percent'
    ]


    gravitational_impact = (net_gravitational_force / max_potential_force) * 100 if max_potential_force > 0 else 0


    for col in final_columns:
        if col not in final_filtered_df.columns:
            final_filtered_df[col] = np.nan

    processed_data_df = final_filtered_df[final_columns].copy()

    return processed_data_df, net_gravitational_force, max_potential_force, gravitational_impact


In [47]:
## ---------- MODIFIED: Run App ---------------
min_nodes = 10
max_nodes = 50
threshold_percent = 0.9

# User input Ticker
source_ticker = 'MSFT'

# Process the data for the network diagram
processed_data_df, net_gravitational_force, max_potential_force, gravitational_impact = process_and_score_stocks(
    six_month_spearman_lagged_correlations,
    three_month_spearman_lagged_correlations,
    screener_data_df,
    source_ticker,
    min_nodes,
    max_nodes,
    threshold_percent,
    )

print(f"Net Gravitational Force: {net_gravitational_force:.2f}")
print(f"Max Potential Gravitational Force: {max_potential_force:.2f}")
print(f"Net Gravitaional Impact: {gravitational_impact:.2f}%")
print('----------------------------------')
processed_data_df

Net Gravitational Force: -470.57
Max Potential Gravitational Force: 803.18
Net Gravitaional Impact: -58.59%
----------------------------------


,source,target,Daily Change,six_month_spearman_correlation,three_month_spearman_correlation,unified_correlation,Orbital Radius,Market Cap,Planet Radius,market_cap_influence,gravitational_force,signed_gravitational_force,gravitational_percent
142,MSFT,NVDA,-2.089655,0.724258,0.860867,0.806223,0.659450,3.462308e+12,1.000000,1.000000,80.622343,-80.622343,-13.689936
23,MSFT,AVGO,-2.878119,0.833271,0.947155,0.901601,1.000000,1.169753e+12,0.610357,0.814110,73.400284,-73.400284,-12.463607
121,MSFT,META,-1.512923,0.469612,0.865892,0.707380,0.306527,1.716961e+12,0.748155,0.879851,62.238861,-62.238861,-10.568361
14,MSFT,AMZN,-0.534609,0.455299,0.778635,0.649301,0.099153,2.251739e+12,0.845516,0.926299,60.144682,-60.144682,-10.212763
184,MSFT,TSLA,1.942904,0.635239,0.816625,0.744071,0.437532,1.047810e+12,0.570827,0.795251,59.172325,59.172325,10.047653
91,MSFT,GOOGL,-0.586227,0.434325,0.774370,0.638352,0.060059,2.139009e+12,0.827074,0.917501,58.568837,-58.568837,-9.945179
90,MSFT,GOOG,-0.615924,0.421631,0.754797,0.621531,0.000000,2.139017e+12,0.827076,0.917502,57.025565,-57.025565,-9.683126
135,MSFT,NFLX,-0.237031,0.477429,0.898013,0.729779,0.386504,5.158559e+11,0.316378,0.673860,49.176894,-49.176894,-8.350397
102,MSFT,INTU,-1.596168,0.753243,0.930750,0.859747,0.850558,2.137322e+11,0.000000,0.522923,44.958130,-44.958130,-7.634037
20,MSFT,ASML,-3.206522,0.628377,0.824583,0.746100,0.444779,3.061686e+11,0.129054,0.584492,43.608946,-43.608946,-7.404941


## Create Network Diagram

In [52]:
# --- MODIFIED: Network Diagram Plotting Function ---
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import random # Added random for shuffling node positions
import matplotlib.patheffects as path_effects # Import for text outline

def plot_ticker_network_visual(source_ticker, processed_data_df, prediction_score, screener_data_df):
    """
    Plots a network graph for a specific ticker and its most correlated partners.
    Includes a detailed information box for the source ticker.
    All data is sourced from the processed_data_df.
    """
    # Filter for connections related to the source ticker
    ticker_connections = processed_data_df[processed_data_df['source'] == source_ticker].copy()

    if ticker_connections.empty:
        print(f"No connections found for ticker {source_ticker} as the source within the provided data.")
        return

    # Filter out any potential self-loops that might have slipped through, although they should be handled by process_and_score_stocks
    ticker_connections = ticker_connections[ticker_connections['source'] != ticker_connections['target']].copy()

    # Ensure the edge attributes list matches the columns available in ticker_connections
    # Corrected based on the expected columns from process_and_score_stocks
    edge_attributes = [
        'Orbital Radius',
        'Planet Radius',
        'normalized_prediction_score',
        'Daily Change',
        'Market Cap',
        'six_month_spearman_correlation',
        'three_month_spearman_correlation',
    ]

    # Create the graph
    G = nx.from_pandas_edgelist(ticker_connections,
                                'source', 'target',
                                edge_attributes)

    # Add the source ticker node explicitly if it's not already added as a target
    if source_ticker not in G.nodes():
        G.add_node(source_ticker)

    # Use subplots to have more control over the layout (graph on left, info on right)
    fig, (ax_main, ax_info) = plt.subplots(1, 2, figsize=(12, 8), gridspec_kw={'width_ratios': [3, 1]})
    fig.patch.set_facecolor('black')
    ax_main.set_facecolor('black')
    ax_info.set_facecolor('black')

    # --- Reticle and Orbits on the main axis ---
    reticle_color = 'gray'
    ax_main.axhline(0, color=reticle_color, linestyle='-', linewidth=0.7)
    ax_main.axvline(0, color=reticle_color, linestyle='-', linewidth=0.7)

    radii = {'Strong': 0.4, 'Moderate': 0.8, 'Weak': 1.2}
    tick_length = 0.025

    for strength, r in radii.items():
        ax_main.plot([-tick_length, tick_length], [r, r], color=reticle_color, linewidth=0.7)
        ax_main.plot([-tick_length, tick_length], [-r, -r], color=reticle_color, linewidth=0.7)
        ax_main.plot([r, r], [-tick_length, tick_length], color=reticle_color, linewidth=0.7)
        ax_main.plot([-r, -r], [-tick_length, tick_length], color=reticle_color, linewidth=0.7)

    # Add labels for the reticle just below the tick marks on the positive x-axis
    label_offset_y = 0.05
    ax_main.text(radii['Strong'], -label_offset_y, 'Strong\nCorrelation', color='white', ha='center', va='top', fontsize=6)
    ax_main.text(radii['Moderate'], -label_offset_y, 'Moderate\nCorrelation', color='white', ha='center', va='top', fontsize=6)
    ax_main.text(radii['Weak'], -label_offset_y, 'Weak\nCorrelation', color='white', ha='center', va='top', fontsize=6)


    pos = {}
    pos[source_ticker] = (0, 0)

    actual_target_connections = ticker_connections[ticker_connections['target'] != source_ticker]
    num_connections = len(actual_target_connections)

    if num_connections > 0:
        angles = [i * (2 * np.pi / num_connections) for i in range(num_connections)]
        random.shuffle(angles)

        # Calculate min and max orbital radius from the filtered connections
        min_orbital_radius = actual_target_connections['Orbital Radius'].min()
        max_orbital_radius = actual_target_connections['Orbital Radius'].max()
        # Handle the case where all orbital radii are the same to avoid division by zero
        radius_range = max_orbital_radius - min_orbital_radius if max_orbital_radius != min_orbital_radius else 1.0


        for i, target_node_name in enumerate(list(actual_target_connections['target'])):
            row = actual_target_connections[actual_target_connections['target'] == target_node_name].iloc[0]
            orbital_radius = row['Orbital Radius']

            # Use the already scaled orbital radius from processed_data_df
            scaled_orbital_radius = orbital_radius # This is now the value between 0 and 1

            min_distance = 0.3
            current_orbital_radius = scaled_orbital_radius + min_distance


            orbit_patch = plt.Circle((0, 0), current_orbital_radius, fill=False, linestyle='--', edgecolor=reticle_color, linewidth=0.7, alpha=0.6)
            ax_main.add_patch(orbit_patch)

            angle = angles[i]
            x = current_orbital_radius * np.cos(angle)
            y = current_orbital_radius * np.sin(angle)
            pos[target_node_name] = (x, y)

    # --- Node Sizing and Coloring ---
    min_planet_size, max_planet_size = 0.05, 1.5
    epsilon = 1e-9

    # Get min/max market caps from the processed_data_df for consistent scaling within this plot
    # Include the source ticker's market cap for scaling if it's not already a target
    all_market_caps = processed_data_df['Market Cap'].tolist()
    source_info_row = processed_data_df[processed_data_df['source'] == source_ticker]
    source_market_cap_for_scaling = source_info_row['Market Cap'].iloc[0] if not source_info_row.empty else None

    if source_market_cap_for_scaling is not None:
         all_market_caps.append(source_market_cap_for_scaling)

    global_min_market_cap = min(all_market_caps) if all_market_caps else epsilon
    global_max_market_cap = max(all_market_caps) if all_market_caps else epsilon

    log_min_mc, log_max_mc = np.log(max(global_min_market_cap, epsilon)), np.log(max(global_max_market_cap, epsilon))


    # Calculate source node size using its market cap from processed_data_df
    source_market_cap = source_info_row['Market Cap'].iloc[0] if not source_info_row.empty else epsilon
    source_market_cap = max(source_market_cap, epsilon)
    log_source_mc = np.log(source_market_cap)
    normalized_source_mc = 0.0 if log_max_mc == log_min_mc else (log_source_mc - log_min_mc) / (log_max_mc - log_min_mc)
    source_planet_radius = min_planet_size + (normalized_source_mc * (max_planet_size - min_planet_size))
    source_node_size = 300 + (source_planet_radius * 500)

    node_sizes_dict = {node: 300 + (ticker_connections[ticker_connections['target'] == node]['Planet Radius'].values[0] * 500) if node != source_ticker else source_node_size for node in G.nodes()}
    node_sizes_list = [node_sizes_dict.get(node, 300) for node in G.nodes()]

    # --- Use normalized_weight from processed_data_df for coloring ---
    colors = ['#820000', '#FF0000', '#FFCC00', '#CB9661', '#00B050', '#0E77FF', '#FFC1F6']
    cmap = mcolors.LinearSegmentedColormap.from_list('weight_cmap', colors)

    # Determine min/max weights for normalization to the colormap range (-1 to 1)
    all_weights = processed_data_df['normalized_prediction_score'].values
    min_weight, max_weight = np.min(all_weights), np.max(all_weights)
    norm_weights = mcolors.Normalize(vmin=min_weight, vmax=max_weight)

    node_colors_list = []
    # Ensure prediction_score is a scalar float
    prediction_score_scalar = float(prediction_score)
    for node in G.nodes():
        if node == source_ticker:
             # Color the source node based on the overall prediction score
            # Handle the case where prediction_score_scalar is 0 to avoid zero range in normalization
            if prediction_score_scalar == 0:
                norm_source = mcolors.Normalize(vmin=np.min(processed_data_df['prediction_score'].values), vmax=np.max(processed_data_df['prediction_score'].values))
            else:
                # Normalize based on the min/max prediction scores
                norm_source = mcolors.Normalize(vmin=np.min(processed_data_df['prediction_score'].values), vmax=np.max(processed_data_df['prediction_score'].values))

            source_node_color = cmap(norm_source(prediction_score_scalar))
            node_colors_list.append(source_node_color)
        else:
            # Color target nodes based on their signed normalized weight
            weight = processed_data_df[(processed_data_df['source'] == source_ticker) & (processed_data_df['target'] == node)]['normalized_prediction_score'].values
            if weight.size > 0:
                node_colors_list.append(cmap(norm_weights(weight[0])))
            else:
                # Default color if weight is not found (shouldn't happen with correct filtering)
                node_colors_list.append('gray')


    # --- Draw Aura and Nodes on the main axis ---
    aura_base_radius = np.sqrt(source_node_size / 300) * 0.05
    # Draw auras only for the source node
    for i in range(1, 15):
        aura = plt.Circle(pos[source_ticker], aura_base_radius * (1 + i * 0.07), color=source_node_color, alpha=0.3 * (1 - (i / 15)))
        ax_main.add_patch(aura)

    nx.draw_networkx_nodes(G, pos, node_size=node_sizes_list, node_color=node_colors_list, alpha=1.0, linewidths=0, ax=ax_main)

    # --- Draw Labels on the main axis ---
    node_color_map = dict(zip(G.nodes(), node_colors_list))
    for node in G.nodes():
        if node in pos:
            node_x, node_y = pos[node]
            node_color = node_color_map[node]
            txt = ax_main.text(node_x, node_y, str(node), color='white', fontweight='bold', fontsize=8, ha='center', va='center')
            txt.set_path_effects([path_effects.Stroke(linewidth=2.5, foreground=node_color), path_effects.Normal()])

    # Fetch company name for the title from screener_data_df
    source_screener_info_for_title = screener_data_df[screener_data_df['code'] == source_ticker]
    company_name = source_screener_info_for_title['name'].iloc[0] if not source_screener_info_for_title.empty and 'name' in source_screener_info_for_title.columns else source_ticker
    ax_main.set_title(f'Network for {company_name} and Most Correlated Partners', color='white', fontsize=11, fontweight='bold')
    ax_main.axis('off')
    ax_main.set_xlim(-1.5, 1.5)
    ax_main.set_ylim(-1.5, 1.5)

    # --- Create Information Box on the right axis ---
    # Get source ticker info from screener_data_df
    source_screener_info = screener_data_df[screener_data_df['code'] == source_ticker].iloc[0] if not screener_data_df[screener_data_df['code'] == source_ticker].empty else None
    source_processed_info = processed_data_df[processed_data_df['source'] == source_ticker].iloc[0] if not processed_data_df[processed_data_df['source'] == source_ticker].empty else None


    if source_screener_info is None or source_processed_info is None:
         print(f"Warning: Source ticker {source_ticker} not found in necessary dataframes for info box.")
         info_text = "Information not available."
         company_name_info = source_ticker # Use ticker as name if not found
    else:
        # Determine prediction score text
        if prediction_score >= 1.0:
            score_meaning = "Strong Increase Signal"
        elif prediction_score > 0.8:
            score_meaning = "Increase Signal"
        elif prediction_score > 0.5:
            score_meaning = "Weak Increase Signal"
        elif prediction_score <= -1.0:
            score_meaning = "Strong Decrease Signal"
        elif prediction_score > 0.9:
            score_meaning = "Increase Signal"
        elif prediction_score < -0.5:
            score_meaning = "Weak Decrease Signal"
        else:
            score_meaning = "Neutral/Mixed Signals"

        # Format Market Cap for readability (using Market Cap from processed_data_df)
        market_cap = source_processed_info['Market Cap']
        if market_cap > 1_000_000_000_000:
            market_cap_str = f"${market_cap/1_000_000_000_000:.2f}T"
        elif market_cap > 1_000_000_000:
            market_cap_str = f"${market_cap/1_000_000_000:.2f}B"
        else:
            market_cap_str = f"${market_cap/1_000_000:.2f}M"

        # Assuming 'Daily Change' is in percentage in processed_data_df
        daily_change_percent = source_processed_info['Daily Change']

        # Format the information string, using screener_data_df for specific fields
        info_text = (
            # Divide prediction_score_scalar by 100 for display as percentage
            f"Gravitational Impact: {gravitational_impact} ({score_meaning})\n"
            f"Total Potential Gravitational Force: {max_potential_force:.2f}\n"
            f"Net Gravitational Force: {net_gravitational_force:.2f}\n
            f"Code: {source_ticker}\n"
            # Get Country, Exchange, Sector, Industry, Type from screener_data_df
            f"Country: {source_screener_info.get('Country', 'N/A')}\n"
            f"Exchange: {source_screener_info.get('Exchange', 'N/A')}\n"
            f"sector: {source_screener_info.get('sector', 'N/A')}\n"
            f"industry: {source_screener_info.get('industry', 'N/A')}\n"
            f"Type: {source_screener_info.get('Type', 'N/A')}\n"
            f"Market Cap: {market_cap_str}\n"
            # Assuming adjusted_close and avgvol_1d are in screener_data_df
            f"Adjusted Close: ${source_screener_info.get('adjusted_close', 0):.2f}\n"
            f"avgvol_1d: {int(source_screener_info.get('avgvol_1d', 0)):,}"
        )
        # Corrected access to the 'name' attribute from the Series
        company_name_info = source_screener_info['name'] if 'name' in source_screener_info and not source_screener_info.empty else source_ticker


    # Add the text to the info axis, adjusting the y position to reduce the gap
    ax_info.text(0.05, 0.95, f"{company_name_info} ({source_ticker})", transform=ax_info.transAxes, fontsize=10, fontweight='bold', color='white', ha='left', va='top')
    ax_info.text(0.05, 0.92, info_text, transform=ax_info.transAxes, fontsize=9, color='white', ha='left', va='top', linespacing=1.5)
    ax_info.axis('off')

    # --- ADDED BACK: Add Colorbar Legend at the bottom of the main graph axis ---
    cbar_ax = fig.add_axes([0.145, 0.08, 0.4, 0.02]) # Adjusted position [left, bottom, width, height]
    # Use the same colormap and normalization as used for the target nod
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm_weights)
    cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')

    # Set the ticks and labels for the colorbar based on the range of normalized weights
    cbar.set_ticks(np.linspace(min_weight, max_weight, 7))
    # Generate labels that represent the range of weights, formatted as percentages
    labels = [f'{val:.1f}%' for val in np.linspace(min_weight, max_weight, 7)]
    cbar.set_ticklabels(labels)

    # Style the colorbar labels
    cbar.ax.tick_params(colors='white', labelsize=6)


    plt.tight_layout(rect=[0, 0.1, 1, 1]) # Adjust layout to make room for colorbar
    plt.show()

## ---------- MODIFIED: Run App ---------------
min_nodes = 10
max_nodes = 50
threshold_percent = 0.9

# User input Ticker
source_ticker = 'CELH'

# Process the data for the network diagram
processed_data_df, prediction_score = process_and_score_stocks(
    unified_correlation_df,
    six_month_spearman_lagged_correlations,
    three_month_spearman_lagged_correlations,
    screener_data_df,
    source_ticker,
    min_nodes,
    max_nodes,
    threshold_percent,
    min_planet_size=0.05,
    max_planet_size=1.5
    )

# Pass the score and the prediction_score_df to the plotting function
plot_ticker_network_visual(source_ticker, processed_data_df, prediction_score, screener_data_df)

SyntaxError: unterminated string literal (detected at line 245) (<ipython-input-52-1089638816>, line 245)

##List of Top Predictions

In [48]:
# prompt: Generate a list of the top gravitational_impact values for all tickers in unified_correlation_df by running each ticker through the function process_and_score_stocks. This should be a table of the ticker, net_gravitational_force:.2f, max_potential_force:.2f, and the gravitational_impact:.2f

import pandas as pd
top_gravitational_impacts = []

# Ensure unified_correlation_df is created from six_month_spearman_lagged_correlations
# assuming unified_correlation_df is meant to represent the base correlations
# based on the usage in the provided code for process_and_score_stocks.
# If unified_correlation_df should be different, please adjust this.
unified_correlation_df = six_month_spearman_lagged_correlations.copy()


# Iterate through each ticker in the unified_correlation_df
for ticker in tqdm(unified_correlation_df.index, desc="Processing tickers for gravitational impact"):
    try:
        # Run each ticker through the process_and_score_stocks function
        # Note: The function previously returned processed_data_df and prediction_score.
        # It now returns processed_data_df, net_gravitational_force, max_potential_force, gravitational_impact
        # We need to unpack the new return values.
        processed_df, net_force, max_potential, gravitational_impact_score = process_and_score_stocks(
            six_month_spearman_lagged_correlations, # Pass the 6-month correlation
            three_month_spearman_lagged_correlations, # Pass the 3-month correlation
            screener_data_df,
            ticker, # Use the current ticker as source_ticker
            min_nodes,
            max_nodes,
            threshold_percent,
        )

        # Append the results to the list
        top_gravitational_impacts.append({
            'ticker': ticker,
            'net_gravitational_force': net_force,
            'max_potential_force': max_potential,
            'gravitational_impact': gravitational_impact_score
        })
    except Exception as e:
        print(f"Error processing ticker {ticker}: {e}")

# Create a DataFrame from the results
gravitational_impact_df = pd.DataFrame(top_gravitational_impacts)

# Sort by gravitational_impact in descending order
gravitational_impact_df = gravitational_impact_df.sort_values(by='gravitational_impact', ascending=False).reset_index(drop=True)

# Display the table
print("Top Gravitational Impact Tickers:")
print(gravitational_impact_df[['ticker', 'net_gravitational_force', 'max_potential_force', 'gravitational_impact']].to_string(index=False, formatters={'net_gravitational_force': '{:.2f}'.format, 'max_potential_force': '{:.2f}'.format, 'gravitational_impact': '{:.2f}'.format}))


Processing tickers for gravitational impact:   0%|          | 0/262 [00:00<?, ?it/s]

Top Gravitational Impact Tickers:
ticker net_gravitational_force max_potential_force gravitational_impact
   KHC                  -89.04              605.63               -14.70
   PPC                 -105.55              633.87               -16.65
  MMYT                 -135.41              807.48               -16.77
  CPRT                  -76.19              406.81               -18.73
  ICLR                 -104.95              558.07               -18.81
   AUR                 -100.46              531.87               -18.89
  ARGX                 -114.91              580.18               -19.81
  ERIE                 -131.21              653.36               -20.08
  TMUS                 -146.80              705.86               -20.80
  ACGL                 -133.94              620.21               -21.60
   DPZ                 -116.57              515.07               -22.63
   COO                 -148.98              645.49               -23.08
   KDP                 -150.25

#Dash App

In [ ]:
# Install necessary libraries (run this cell first)
!pip install -q dash dash-bootstrap-components plotly

In [ ]:
from dash import Dash, html # Import the main Dash class to create our application.
import os # Import os to set environment variables for ngrok.
from pyngrok import ngrok # Import ngrok from pyngrok to create a public tunnel.
from google.colab import userdata # Import userdata to access Colab secrets

# --- Configure Ngrok for public access ---
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
os.environ["NGROK_AUTH_TOKEN"] = NGROK_AUTH_TOKEN

# Define the port your Dash app will run on.
DASH_PORT = 8050

# Create a public ngrok tunnel to the specified port. This will generate a
# public URL that you can access from anywhere, including your iPad, bypassing Colab's iframe restrictions.
print("Establishing Ngrok tunnel...")
public_url = ngrok.connect(DASH_PORT)
print(f"Your Dash app will be accessible at: {public_url}")

# --- Initialize the Dash application ---
app = Dash(__name__)

# --- Define the app layout ---
app.layout = html.Div([
    html.H1("Welcome to the Finacial Observatory!", style={'textAlign': 'center', 'color': 'blue'}),
    html.P("I love you, Nix!", style={'textAlign': 'center', 'color': 'green'})
])

# --- Run the Dash server ---
print(f"Starting Dash server on port {DASH_PORT}...")
app.run(debug=True, port=DASH_PORT)

# Important Note:
# Keep this Colab notebook cell running and active for your app to remain live.
# If the Colab runtime disconnects or the cell execution stops, your app will go offline.

Establishing Ngrok tunnel...
Your Dash app will be accessible at: NgrokTunnel: "https://0456-34-83-174-46.ngrok-free.app" -> "http://localhost:8050"
Starting Dash server on port 8050...


<IPython.core.display.Javascript object>

In [ ]:
from pyngrok import ngrok

# Kills all ngrok processes that pyngrok started
print("Attempting to kill all active ngrok tunnels...")
ngrok.kill()
print("All ngrok tunnels should now be terminated.")

Attempting to kill all active ngrok tunnels...
All ngrok tunnels should now be terminated.


In [ ]:
# --- Import necessary libraries ---
from dash import Dash, html, dcc, Input, Output # We need dcc for interactive components and Input/Output for callbacks
import os
from pyngrok import ngrok
from google.colab import userdata

# --- Configure Ngrok for public access ---
# This part remains the same to create a public URL for your app
try:
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    os.environ["NGROK_AUTH_TOKEN"] = NGROK_AUTH_TOKEN
except userdata.SecretNotFoundError:
    print("NGROK_AUTH_TOKEN not found in Colab secrets. Please add it.")
    # You might want to handle this case, e.g., by exiting or using a default token if applicable
    exit()


# Define the port your Dash app will run on.
DASH_PORT = 8051

# Create a public ngrok tunnel to the specified port.
print("Establishing Ngrok tunnel...")
public_url = ngrok.connect(DASH_PORT)
print(f"🚀 Your Dash app will be accessible at: {public_url}")

# --- Initialize the Dash application ---
app = Dash(__name__)

# --- Define the app layout ---
# The layout is what the user sees. We've added an input box and a div for the output.
app.layout = html.Div([
    html.H1("Simple Interactive App", style={'textAlign': 'center', 'color': '#4A90E2'}),

    html.P("Type something in the box below:", style={'textAlign': 'center'}),

    # dcc.Input is an interactive component that allows user text input.
    dcc.Input(
        id='user-input-box',      # A unique ID to identify this component in callbacks
        type='text',               # The type of input (text, number, password, etc.)
        placeholder='Enter your message here...', # Placeholder text
        style={'width': '50%', 'padding': '10px', 'margin': 'auto', 'display': 'block'}
    ),

    # This html.Div will be used to display the output from our callback.
    # It starts empty.
    html.Div(
        id='app-output-div',
        style={'textAlign': 'center', 'marginTop': '20px', 'fontSize': '24px', 'color': '#50E3C2'}
    )
])

# --- Define the Callback ---
# This is the core of the app's interactivity.
# It connects the input component to the output component.
@app.callback(
    Output('app-output-div', 'children'), # The component property we want to update (the content of the div)
    Input('user-input-box', 'value')      # The component property that triggers the callback (the value of the input box)
)
def update_output_div(input_value):
    """
    This function is called automatically by Dash whenever the 'value' of the 'user-input-box' changes.
    The new value is passed as the 'input_value' argument.
    The function's return value is used to update the 'children' of the 'app-output-div'.
    """
    if input_value is None or input_value == '':
        return "Type something to get a response!"
    else:
        # The f-string creates a new string with the user's input.
        return f'The app says: "{input_value}"'

# --- Run the Dash server ---
print(f"Starting Dash server on port {DASH_PORT}...")
# The host='0.0.0.0' is important for running in environments like Colab
app.run(debug=True, port=DASH_PORT)

# Important Note:
# Keep this Colab notebook cell running and active for your app to remain live.
# If the Colab runtime disconnects or the cell execution stops, your app will go offline.


Establishing Ngrok tunnel...
🚀 Your Dash app will be accessible at: NgrokTunnel: "https://bf60-34-83-174-46.ngrok-free.app" -> "http://localhost:8051"
Starting Dash server on port 8051...


<IPython.core.display.Javascript object>

In [ ]:
# Display descriptive statistics of the calculated scores
print("Descriptive statistics of prediction scores:")
display(prediction_df['score'].describe())

# Display the first few rows of the prediction_df to see sample scores
print("\nSample of calculated prediction scores:")
display(prediction_df.head())

# Check the maximum score to see if it's close to the threshold
max_score = prediction_df['score'].max()
print(f"\nMaximum prediction score: {max_score:.2f}")

# Check how many scores are above 95
count_above_95 = (prediction_df['score'] > 95).sum()
print(f"\nNumber of tickers with score > 95: {count_above_95}")

Descriptive statistics of prediction scores:


,score
count,261.000000
mean,-41.240832
std,29.504366
min,-100.000000
25%,-62.365173
50%,-40.785424
75%,-23.796721
max,44.134355



Sample of calculated prediction scores:


,ticker,score
0,AAPL,-2.079788
1,ABNB,-73.324077
2,ACGL,1.388236
3,ADBE,-70.945344
4,ADI,-83.863688



Maximum prediction score: 44.13

Number of tickers with score > 95: 0


In [ ]:
# Pass the score and the processed_data_df to the plotting function
plot_ticker_network_visual(source_ticker, processed_data_df, prediction_score)

NetworkXError: Invalid edge_attr argument: ['unified_correlation', 'Orbital Radius', 'Planet Radius', 'normalized_prediction_score', 'Daily Change', 'Market Cap', 'six_month_spearman_correlation', 'three_month_spearman_correlation', 'coefficient_of_determination']

In [55]:
# --- MODIFIED: Network Diagram Plotting Function ---
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import random # Added random for shuffling node positions
import matplotlib.patheffects as path_effects # Import for text outline

def plot_ticker_network_visual(source_ticker, processed_data_df, net_gravitational_force, max_potential_force, screener_data_df):
    """
    Plots a network graph for a specific ticker and its most correlated partners.
    Includes a detailed information box for the source ticker.
    Takes the net_gravitational_force (scalar), max_potential_force (scalar), and the processed_data_df.
    All data is sourced from the processed_data_df.
    """
    # Filter for connections related to the source ticker from the processed_data_df
    # Note: processed_data_df should *not* include the source ticker as a target of itself
    ticker_connections = processed_data_df[processed_data_df['source'] == source_ticker].copy()


    if ticker_connections.empty:
        print(f"No connections found for ticker {source_ticker} as the source within the provided processed data.")
        # Try to get source info even if no connections for info box
        source_screener_info = screener_data_df[screener_data_df['code'] == source_ticker].iloc[0] if not screener_data_df[screener_data_df['code'] == source_ticker].empty else None
        if source_screener_info is None:
             print(f"Source ticker {source_ticker} not found in screener data.")
             return
        else:
            # Create a graph with just the source node if no connections
            G = nx.Graph()
            G.add_node(source_ticker)
            pos = {source_ticker: (0,0)}
            node_sizes_list = [500] # Default size for single node
            node_colors_list = ['gray'] # Default color
            company_name = source_screener_info['name'] if 'name' in source_screener_info and not source_screener_info.empty else source_ticker
            info_text = "No significant correlated connections found.\n"
            # Determine prediction score text based on net_gravitational_force (which will be 0 in this case)
            impact_score = 0
            score_meaning = "Neutral/Mixed Signals" # Default if no force

            # Format Market Cap for readability
            market_cap = source_screener_info['market_capitalization'] if 'market_capitalization' in source_screener_info else 0
            if market_cap > 1_000_000_000_000:
                market_cap_str = f"${market_cap/1_000_000_000_000:.2f}T"
            elif market_cap > 1_000_000_000:
                market_cap_str = f"${market_cap/1_000_000_000:.2f}B"
            else:
                market_cap_str = f"${market_cap/1_000_000:.2f}M"


            info_text += (
                 f"Prediction Score: {impact_score:.1f}% ({score_meaning})\n"
                 f"Code: {source_ticker}\n"
                 f"Country: {source_screener_info.get('Country', 'N/A')}\n"
                 f"Exchange: {source_screener_info.get('Exchange', 'N/A')}\n"
                 f"sector: {source_screener_info.get('sector', 'N/A')}\n"
                 f"industry: {source_screener_info.get('industry', 'N/A')}\n"
                 f"Type: {source_screener_info.get('Type', 'N/A')}\n"
                 f"Market Cap: {market_cap_str}\n"
                 f"Adjusted Close: ${source_screener_info.get('adjusted_close', 0):.2f}\n"
                 f"avgvol_1d: {int(source_screener_info.get('avgvol_1d', 0)):,}"
            )


            fig, (ax_main, ax_info) = plt.subplots(1, 2, figsize=(12, 8), gridspec_kw={'width_ratios': [3, 1]})
            fig.patch.set_facecolor('black')
            ax_main.set_facecolor('black')
            ax_info.set_facecolor('black')

            # Draw the single source node
            nx.draw_networkx_nodes(G, pos, node_size=node_sizes_list, node_color=node_colors_list, alpha=1.0, linewidths=0, ax=ax_main)
            txt = ax_main.text(0, 0, str(source_ticker), color='white', fontweight='bold', fontsize=8, ha='center', va='center')
            txt.set_path_effects([path_effects.Stroke(linewidth=2.5, foreground=node_colors_list[0]), path_effects.Normal()])

            ax_main.set_title(f'Network for {company_name} and Most Correlated Partners', color='white', fontsize=11, fontweight='bold')
            ax_main.axis('off')
            ax_main.set_xlim(-1.5, 1.5)
            ax_main.set_ylim(-1.5, 1.5)


            # Add the text to the info axis
            ax_info.text(0.05, 0.95, f"{company_name} ({source_ticker})", transform=ax_info.transAxes, fontsize=10, fontweight='bold', color='white', ha='left', va='top')
            ax_info.text(0.05, 0.92, info_text, transform=ax_info.transAxes, fontsize=9, color='white', ha='left', va='top', linespacing=1.5)
            ax_info.axis('off')

            plt.tight_layout(rect=[0, 0.1, 1, 1]) # Adjust layout
            plt.show()

            return # Exit the function here if no connections


    # Filter out any potential self-loops that might have slipped through, although they should be handled by process_and_score_stocks
    ticker_connections = ticker_connections[ticker_connections['source'] != ticker_connections['target']].copy()


    # Ensure the edge attributes list matches the columns available in ticker_connections
    edge_attributes = [
        'unified_correlation',
        'Orbital Radius',
        'Planet Radius',
        'signed_gravitational_force',
        'Daily Change',
        'Market Cap',
        'six_month_spearman_correlation',
        'three_month_spearman_correlation',
        'gravitational_force',
        'gravitational_force_impact'
    ]

    # Create the graph using only the target connections
    G = nx.from_pandas_edgelist(ticker_connections,
                                'source', 'target',
                                edge_attributes)

    # Add the source ticker node explicitly at the center (0,0)
    G.add_node(source_ticker)

    # Use subplots to have more control over the layout (graph on left, info on right)
    fig, (ax_main, ax_info) = plt.subplots(1, 2, figsize=(12, 8), gridspec_kw={'width_ratios': [3, 1]})
    fig.patch.set_facecolor('black')
    ax_main.set_facecolor('black')
    ax_info.set_facecolor('black')

    # --- Reticle and Orbits on the main axis ---
    reticle_color = 'gray'
    ax_main.axhline(0, color=reticle_color, linestyle='-', linewidth=0.7)
    ax_main.axvline(0, color=reticle_color, linestyle='-', linewidth=0.7)

    radii = {'Strong': 0.4, 'Moderate': 0.8, 'Weak': 1.2}
    tick_length = 0.025

    for strength, r in radii.items():
        ax_main.plot([-tick_length, tick_length], [r, r], color=reticle_color, linewidth=0.7)
        ax_main.plot([-tick_length, tick_length], [-r, -r], color=reticle_color, linewidth=0.7)
        ax_main.plot([r, r], [-tick_length, tick_length], color=reticle_color, linewidth=0.7)
        ax_main.plot([-r, -r], [-tick_length, tick_length], color=reticle_color, linewidth=0.7)

    # Add labels for the reticle just below the tick marks on the positive x-axis
    label_offset_y = 0.05
    ax_main.text(radii['Strong'], -label_offset_y, 'Strong\nCorrelation', color='white', ha='center', va='top', fontsize=6)
    ax_main.text(radii['Moderate'], -label_offset_y, 'Moderate\nCorrelation', color='white', ha='center', va='top', fontsize=6)
    ax_main.text(radii['Weak'], -label_offset_y, 'Weak\nCorrelation', color='white', ha='center', va='top', fontsize=6)


    pos = {}
    pos[source_ticker] = (0, 0) # Source ticker is always at the center

    num_connections = len(ticker_connections)

    if num_connections > 0:
        angles = [i * (2 * np.pi / num_connections) for i in range(num_connections)]
        random.shuffle(angles)

        # Calculate min and max orbital radius from the filtered connections
        min_orbital_radius = ticker_connections['Orbital Radius'].min()
        max_orbital_radius = ticker_connections['Orbital Radius'].max()
        # Handle the case where all orbital radii are the same to avoid division by zero
        radius_range = max_orbital_radius - min_orbital_radius if max_orbital_radius != min_orbital_radius else 1.0


        for i, target_node_name in enumerate(list(ticker_connections['target'])):
            row = ticker_connections[ticker_connections['target'] == target_node_name].iloc[0]
            orbital_radius = row['Orbital Radius']

            # Use the already scaled orbital radius from processed_data_df
            scaled_orbital_radius = orbital_radius # This is now the value between 0 and 1

            min_distance = 0.3 # Minimum distance from the center
            current_orbital_radius = scaled_orbital_radius + min_distance


            # Add orbit circle patch only for targets
            orbit_patch = plt.Circle((0, 0), current_orbital_radius, fill=False, linestyle='--', edgecolor=reticle_color, linewidth=0.7, alpha=0.6)
            ax_main.add_patch(orbit_patch)

            angle = angles[i]
            x = current_orbital_radius * np.cos(angle)
            y = current_orbital_radius * np.sin(angle)
            pos[target_node_name] = (x, y)

    # --- Node Sizing and Coloring ---
    min_planet_size, max_planet_size = 0.05, 1.5
    epsilon = 1e-9

    # Get min/max market caps from the processed_data_df for consistent scaling within this plot
    # We need the source ticker's market cap to scale it relative to the targets.
    # Get source ticker's market cap from screener_data_df
    source_screener_info = screener_data_df[screener_data_df['code'] == source_ticker].iloc[0] if not screener_data_df[screener_data_df['code'] == source_ticker].empty else None

    if source_screener_info is None:
         print(f"Warning: Source ticker {source_ticker} not found in screener data for node sizing.")
         source_market_cap = epsilon
    else:
         source_market_cap = source_screener_info['market_capitalization'] if 'market_capitalization' in source_screener_info else epsilon


    all_market_caps = ticker_connections['Market Cap'].tolist()
    all_market_caps.append(source_market_cap)

    global_min_market_cap = min(all_market_caps) if all_market_caps else epsilon
    global_max_market_cap = max(all_market_caps) if all_market_caps else epsilon

    log_min_mc, log_max_mc = np.log(max(global_min_market_cap, epsilon)), np.log(max(global_max_market_cap, epsilon))
    log_mc_range = log_max_mc - log_min_mc


    # Calculate source node size using its market cap
    source_market_cap = max(source_market_cap, epsilon) # Ensure cap is not zero
    log_source_mc = np.log(source_market_cap)
    normalized_source_mc = 0.0 if log_mc_range <= epsilon else (log_source_mc - log_min_mc) / log_mc_range
    source_planet_radius = min_planet_size + (normalized_source_mc * (max_planet_size - min_planet_size))
    source_node_size = 300 + (source_planet_radius * 500)


    # Calculate target node sizes using Planet Radius from processed_data_df
    node_sizes_dict = {}
    for index, row in ticker_connections.iterrows():
         target_node = row['target']
         # Scale the Planet Radius (0 to 1) to the desired node size range
         target_planet_radius_scaled = min_planet_size + (row['Planet Radius'] * (max_planet_size - min_planet_size))
         node_sizes_dict[target_node] = 300 + (target_planet_radius_scaled * 500)

    node_sizes_dict[source_ticker] = source_node_size # Add source node size to the dictionary
    node_sizes_list = [node_sizes_dict.get(node, 300) for node in G.nodes()]


    # --- Use gravitational_force_impact from processed_data_df for coloring ---
    colors = ['#820000', '#FF0000', '#FFCC00', '#CB9661', '#00B050', '#0E77FF', '#FFC1F6']
    cmap = mcolors.LinearSegmentedColormap.from_list('impact_cmap', colors)

    # Determine min/max impact for normalization to the colormap range (-100 to 100)
    # Use impact from the target connections only
    all_impacts = ticker_connections['gravitational_force_impact'].values
    # Handle case where all impacts are the same or zero
    min_impact = np.min(all_impacts) if all_impacts.size > 0 else 0
    max_impact = np.max(all_impacts) if all_impacts.size > 0 else 0

    # Create normalization, handle case where min == max
    norm_impacts = mcolors.Normalize(vmin=min_impact, vmax=max_impact) if max_impact != min_impact else mcolors.Normalize(vmin=-1, vmax=1)


    node_colors_list = []

    # Recalculate gravitational_impact percentage here for consistent coloring of source node
    overall_gravitational_impact = (net_gravitational_force / max_potential_force) * 100 if max_potential_force > 0 else 0


    # Color the source node based on the overall gravitational_impact percentage
    # Need to normalize the overall_gravitational_impact to the same range as the target node impacts for consistent coloring.
    # Use the same normalization as for target node impacts
    # Ensure the impact value is within the normalization range
    clamped_overall_impact = np.clip(overall_gravitational_impact, min_impact, max_impact)
    source_node_color = cmap(norm_impacts(clamped_overall_impact))


    for node in G.nodes():
        if node == source_ticker:
            node_colors_list.append(source_node_color)
        else:
            # Color target nodes based on their gravitational force impact
            impact = ticker_connections[ticker_connections['target'] == node]['gravitational_force_impact'].values
            if impact.size > 0:
                 # Ensure impact value is within the normalization range
                 clamped_impact = np.clip(impact[0], min_impact, max_impact)
                 node_colors_list.append(cmap(norm_impacts(clamped_impact)))
            else:
                # Default color if impact is not found (shouldn't happen with correct filtering)
                node_colors_list.append('gray')


    # --- Draw Aura and Nodes on the main axis ---
    aura_base_radius = np.sqrt(source_node_size / 300) * 0.05
    # Draw auras only for the source node
    for i in range(1, 15):
        aura = plt.Circle(pos[source_ticker], aura_base_radius * (1 + i * 0.07), color=source_node_color, alpha=0.3 * (1 - (i / 15)))
        ax_main.add_patch(aura)

    nx.draw_networkx_nodes(G, pos, node_size=node_sizes_list, node_color=node_colors_list, alpha=1.0, linewidths=0, ax=ax_main)

    # --- Draw Edges on the main axis ---
    # Color edges based on signed_gravitational_force
    edge_colors = [G[u][v]['signed_gravitational_force'] for u, v in G.edges()]

    # Normalize edge colors to the impact colormap
    # Use the same normalization as for target node colors for consistency
    norm_edges = norm_impacts # Use the same normalization as node impact


    edge_color_list = [cmap(norm_edges(color)) for color in edge_colors]

    # Set edge width based on absolute gravitational_force (or impact?)
    # Let's use absolute gravitational_force for width
    edge_widths = [G[u][v]['gravitational_force'].abs() for u, v in G.edges()]

    # Scale edge widths
    min_edge_width, max_edge_width = 0.5, 3.0
    min_force = np.min(edge_widths) if edge_widths else 0
    max_force = np.max(edge_widths) if edge_widths else 1 # Avoid division by zero
    force_range = max_force - min_force if max_force > min_force else 1.0

    scaled_edge_widths = [min_edge_width + ( (w - min_force) / force_range ) * (max_edge_width - min_edge_width) for w in edge_widths]


    nx.draw_networkx_edges(G, pos, edge_color=edge_color_list, width=scaled_edge_widths, ax=ax_main, alpha=0.7)


    # --- Draw Labels on the main axis ---
    node_color_map = dict(zip(G.nodes(), node_colors_list))
    for node in G.nodes():
        if node in pos:
            node_x, node_y = pos[node]
            node_color = node_color_map[node]
            txt = ax_main.text(node_x, node_y, str(node), color='white', fontweight='bold', fontsize=8, ha='center', va='center')
            txt.set_path_effects([path_effects.Stroke(linewidth=2.5, foreground=node_color), path_effects.Normal()])

    # Fetch company name for the title from screener_data_df
    source_screener_info_for_title = screener_data_df[screener_data_df['code'] == source_ticker]
    company_name = source_screener_info_for_title['name'].iloc[0] if not source_screener_info_for_title.empty and 'name' in source_screener_info_for_title.columns else source_ticker
    ax_main.set_title(f'Network for {company_name} and Most Correlated Partners', color='white', fontsize=11, fontweight='bold')
    ax_main.axis('off')
    ax_main.set_xlim(-1.5, 1.5)
    ax_main.set_ylim(-1.5, 1.5)

    # --- Create Information Box on the right axis ---
    # Get source ticker info from screener_data_df
    source_screener_info = screener_data_df[screener_data_df['code'] == source_ticker].iloc[0] if not screener_data_df[screener_data_df['code'] == source_ticker].empty else None
    # processed_data_df doesn't contain the source ticker row anymore after the last modification
    # We need net_gravitational_force and max_potential_force from the calling scope
    # Let's assume net_gravitational_force and max_potential_force are passed to the function.


    if source_screener_info is None:
         print(f"Warning: Source ticker {source_ticker} not found in screener_data_df for info box.")
         info_text = "Information not available."
         company_name_info = source_ticker # Use ticker as name if not found
         impact_score = 0
         score_meaning = "Data Missing"
         max_potential_force_display = 0
    else:
        # Determine prediction score text based on the overall gravitational_impact percentage
        # Recalculate gravitational_impact here for the info box
        # We need max_potential_force to be passed into this function
        # Assuming max_potential_force is now a parameter of plot_ticker_network_visual
        impact_score = (net_gravitational_force / max_potential_force) * 100 if max_potential_force > 0 else 0


        if impact_score >= 50:
            score_meaning = "Strong Increase Signal"
        elif impact_score > 10:
            score_meaning = "Increase Signal"
        elif impact_score > 0:
            score_meaning = "Weak Increase Signal"
        elif impact_score <= -50:
            score_meaning = "Strong Decrease Signal"
        elif impact_score < -10:
            score_meaning = "Decrease Signal"
        elif impact_score < 0:
            score_meaning = "Weak Decrease Signal"
        else:
            score_meaning = "Neutral/Mixed Signals"

        # Format Market Cap for readability (using Market Cap from screener_data_df for info box)
        market_cap = source_screener_info.get('market_capitalization', 0)
        if market_cap > 1_000_000_000_000:
            market_cap_str = f"${market_cap/1_000_000_000_000:.2f}T"
        elif market_cap > 1_000_000_000:
            market_cap_str = f"${market_cap/1_000_000_000:.2f}B"
        else:
            market_cap_str = f"${market_cap/1_000_000:.2f}M"

        # Format the information string, using screener_data_df for specific fields
        info_text = (
            f"Gravitational Impact: {impact_score:.1f}% ({score_meaning})\n"
            f"Total Potential Gravitational Force: {max_potential_force:.2f}\n"
            f"Net Gravitational Force: {net_gravitational_force:.2f}\n"
            f"Code: {source_ticker}\n"
            # Get Country, Exchange, Sector, Industry, Type from screener_data_df
            f"Country: {source_screener_info.get('Country', 'N/A')}\n"
            f"Exchange: {source_screener_info.get('Exchange', 'N/A')}\n"
            f"sector: {source_screener_info.get('sector', 'N/A')}\n"
            f"industry: {source_screener_info.get('industry', 'N/A')}\n"
            f"Type: {source_screener_info.get('Type', 'N/A')}\n"
            f"Market Cap: {market_cap_str}\n"
            # Assuming adjusted_close and avgvol_1d are in screener_data_df
            f"Adjusted Close: ${source_screener_info.get('adjusted_close', 0):.2f}\n"
            f"avgvol_1d: {int(source_screener_info.get('avgvol_1d', 0)):,}"
        )
        # Corrected access to the 'name' attribute from the Series
        company_name_info = source_screener_info['name'] if 'name' in source_screener_info and not source_screener_info.empty else source_ticker


    # Add the text to the info axis, adjusting the y position to reduce the gap
    ax_info.text(0.05, 0.95, f"{company_name_info} ({source_ticker})", transform=ax_info.transAxes, fontsize=10, fontweight='bold', color='white', ha='left', va='top')
    ax_info.text(0.05, 0.92, info_text, transform=ax_info.transAxes, fontsize=9, color='white', ha='left', va='top', linespacing=1.5)
    ax_info.axis('off')

    # --- Add Colorbar Legend at the bottom of the main graph axis ---
    cbar_ax = fig.add_axes([0.145, 0.08, 0.4, 0.02]) # Adjusted position [left, bottom, width, height]
    # Use the same colormap and normalization as used for the target node colors
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm_impacts)
    cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')

    # Set the ticks and labels for the colorbar based on the range of impacts
    # Use the actual min/max impacts for labels
    cbar.set_ticks(np.linspace(min_impact, max_impact, 7))
    # Generate labels that represent the range of impacts, formatted as percentages
    labels = [f'{val:.1f}%' for val in np.linspace(min_impact, max_impact, 7)]
    cbar.set_ticklabels(labels)

    # Style the colorbar labels
    cbar.ax.tick_params(colors='white', labelsize=6)


    plt.tight_layout(rect=[0, 0.1, 1, 1]) # Adjust layout to make room for colorbar
    plt.show()

## ---------- MODIFIED: Run App ---------------
min_nodes = 10
max_nodes = 50
threshold_percent = 0.9

# User input Ticker
source_ticker = 'CELH'

# Process the data for the network diagram
# Unpack the returned values into variables
processed_data_df, net_gravitational_force, max_potential_force = process_and_score_stocks(
    six_month_spearman_lagged_correlations,
    three_month_spearman_lagged_correlations,
    screener_data_df,
    source_ticker,
    min_nodes,
    max_nodes,
    threshold_percent,
    # market_cap_offset was removed
    )

# Pass the calculated values to the plotting function
plot_ticker_network_visual(source_ticker, processed_data_df, net_gravitational_force, max_potential_force, screener_data_df)

ValueError: too many values to unpack (expected 3)